### Projeto 2-1 Visão Computacional
Elisa Malzoni

In [1]:
%reset -f
%matplotlib inline

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

import time

In [16]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
           qualityLevel = 0.3,
           minDistance = 7,
           blockSize = 7 )

color = np.random.randint(0,255,(100,3))

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


ret, previous = captura.read()

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)
mask = np.zeros_like(previous)

while(1):
    
    ret, actual = captura.read()
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
        
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # Calcula fluxo otico medio
    # subtrair old-new
    # tirar media da 1a coluna e 2a coluna 
        # esse é o fluxo medio
#     fomedio = np.su
    

    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
    

    frame = cv2.add(actual, mask)
    
    cv2.imshow("Video", frame)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()